# Checking bass onsets

In [1]:
import numpy as np
import pandas as pd

from src import utils

In [2]:
corp = utils.load_corpus_from_files(f'{utils.get_project_root()}/data/cambridge-jazz-trio-database-v02')

In [3]:
def get_between(arr, i1, i2) -> np.array:
    """From an array `arr`, get all onsets between an upper and lower bound `i1` and `i2` respectively"""
    return arr[np.where(np.logical_and(arr >= i1, arr <= i2))]

## Check 'rakes'

We define a 'rake' as three eighth note triplets preceded and followed by a quarter note: see the '3-note rake' on the below image

![bass rake](https://study.com/cimages/multimages/16/walking_bass_example.png)

In [4]:
def process_rake(onsets, b1, b2, b3):
    # Get onsets between first and second beat
    between = get_between(onsets, b1, b3)
    # If we have enough onsets for a three beat rake
    if len(between) == 5:
        between.sort()
        # If beat 1 = onset 1, beat 2 = onset 4
        if all((between[0] == b1, between[1] == b2, between[-1] == b3)):
            # We probably have a rake
            return between

In [7]:
fs = []
total_rakes = 0
for track in corp:
    if not track.item['has_annotations']:
        continue

    # PROCESS AUTOMATIC DETECTIONS
    # Get both the automatic bass beats and onsets
    my_beats = track.summary_dict['bass']
    my_onsets = track.ons['bass']
    # Process automatically detect beats and onsets
    my_rakes = [process_rake(my_onsets, b1, b2, b3) for b1, b2, b3 in zip(my_beats, my_beats[1:], my_beats[2:])]
    # Sort the onsets in each rake
    my_rakes = [sorted(r) for r in my_rakes if r is not None]
    # PROCESS MANUAL DETECTIONS
    # Load in the manually detcted bass onsets
    my_onsets_man = np.loadtxt(f'{utils.get_project_root()}/references/manual_annotation/{track.item["fname"]}_bass.txt', ndmin=1, usecols=0)
    # Get all onsets between the first and last note of each identified rake (with some tolerance)
    my_rakes_man = [get_between(my_onsets_man, rake[0] - 0.05, rake[-1] + 0.05) for rake in my_rakes]
    # If we don't have any rakes, skip the track
    if not all((len(my_rakes) > 0, len(my_rakes_man) > 0)):
        continue
    # Get the number of rakes identified
    total_rakes += len(my_rakes)
    # Join together both arrays
    ref = np.unique(np.concatenate(my_rakes_man))
    onsets = np.unique(np.concatenate(my_rakes))
    # Generate the summary dict and append the results
    res = track.compare_onset_detection_accuracy(ref=ref, onsets=onsets, window=0.05)
    fs.append(res['f_score'])
print(np.mean(fs), np.std(fs), total_rakes)

0.9073807168875347 0.08081973789470916 57


## Check beats with both bass and drums playing together

In [8]:
bdres = []
nbeats = 0
for track in corp:
    if not track.item['has_annotations']:
        continue

    # PROCESS AUTOMATIC DETECTIONS
    # Get automatic detected bass beats when both bass and drums playing on same beat
    beats_auto = (
        pd.DataFrame((track.summary_dict['bass'], track.summary_dict['drums']))
        .transpose()
        .rename(columns={0: 'bass', 1: 'drums'})
        .dropna()
        .reset_index(drop=True)
    )
    my_beats_auto = beats_auto['bass'].values
    nbeats += len(my_beats_auto)

    # PROCESS MANUAL DETECTIONS
    # Load in manually detected bass and drums onsets
    bass_onsets_man = np.loadtxt(f'{utils.get_project_root()}/references/manual_annotation/{track.item["fname"]}_bass.txt', ndmin=1, usecols=0)
    drums_onsets_man = np.loadtxt(f'{utils.get_project_root()}/references/manual_annotation/{track.item["fname"]}_drums.txt', ndmin=1, usecols=0)
    # Match bass and drums onsets to auto detected beats
    bass_matched = track.match_onsets_and_beats(track.summary_dict['beats'], bass_onsets_man)
    drums_matched = track.match_onsets_and_beats(track.summary_dict['beats'], drums_onsets_man)
    # Create the dataframe
    beats_man = (
        pd.DataFrame((bass_matched, drums_matched))
        .transpose()
        .rename(columns={0: 'bass', 1: 'drums'})
        .dropna()
        .reset_index(drop=True)
    )
    my_beats_man = beats_man['bass'].values

    # COMPARE AUTOMATIC AND MANUAL DETECTIONS
    res = track.compare_onset_detection_accuracy(ref=my_beats_man, onsets=my_beats_auto, window=0.05)
    bdres.append(res['f_score'])
print(np.mean(bdres), np.std(bdres), nbeats)

0.924927705501795 0.06104743970455487 10538


## Checking specific beats of the bar

In [9]:
fsbeat = {i: [] for i in range(1, 5)}
for track in corp:
    if not track.item['has_annotations']:
        continue

    # Get automatically detected metre
    bass_metre_auto = np.column_stack((track.ons['metre_auto'], track.summary_dict['bass']))
    # Get manually detected metre
    bass_onsets_man = np.loadtxt(f'{utils.get_project_root()}/references/manual_annotation/{track.item["fname"]}_bass.txt', ndmin=1, usecols=0)
    bass_matched = track.match_onsets_and_beats(track.summary_dict['beats'], bass_onsets_man)
    bass_metre_man = np.column_stack((track.ons['metre_auto'], bass_matched))

    # Iterate through each beat in our time signature
    for i in range(1, track.item['time_signature'] + 1):
        metre_autos = bass_metre_auto[bass_metre_auto[:, 0] == i, 1]
        metre_mans = bass_metre_man[bass_metre_man[:, 0] == i, 1]
        # Remove nan values
        metre_autos = metre_autos[~pd.isnull(metre_autos)]
        metre_mans = metre_mans[~pd.isnull(metre_mans)]
        # Calculate f-score
        res = track.compare_onset_detection_accuracy(ref=metre_mans, onsets=metre_autos, window=0.05)
        fsbeat[i].append(res['f_score'])
for i, vals in fsbeat.items():
    print(i, np.mean(vals), np.std(vals))

1 0.9390512485693824 0.06964151206426002
2 0.9360016788751553 0.11514022287159142
3 0.9469217641316713 0.060315214214627294
4 0.9502508879139175 0.044844926198127806


In [5]:
for instr in ['piano', 'bass', 'drums']:
    print(instr, np.mean([i.item['validation'][instr]['mean_asynchrony'] for i in corp if i.item['validation'][instr] is not None]))

piano -0.005539215217091873
bass -0.004146696717315414
drums -0.0035208749287690445
